In [88]:
from typing import List, Optional, Literal
from pydantic import BaseModel, Field
import os
import pandas as pd
import uuid
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langgraph.graph import StateGraph, END
from typing import TypedDict, Any
from IPython.display import Image, display
import pprint

In [89]:
# --- Helper Models for Quiz Variants ---
class FlashCardData(BaseModel):
    word_hanzi: str
    pinyin: str
    definition_en: str
    definition_vi: str
    example_sentence: Optional[str] = None
    example_pinyin: Optional[str] = None
    example_meaning_en: Optional[str] = None
    example_meaning_vi: Optional[str] = None
    extra_info_md: Optional[str] = None

class QuizOptionData(BaseModel):
    """Used for Single Choice, Multiple Choice, Listening, Image Selection"""
    text: str = Field(..., description="The answer text or label")
    is_correct: bool
    order: int
    image_url: Optional[str] = Field(None, description="URL if the option is an image")

class MatchPairData(BaseModel):
    """Used for Match Pairs"""
    left_text: str
    right_text: str
    order: int

class SentenceQuizData(BaseModel):
    """Used for Fill in the Blank, Order Sentence"""
    chinese_sentence: str
    pinyin_sentence: Optional[str] = None
    correct_answer: Optional[str] = Field(None, description="The specific word for 'Fill in the Blank'")
    distractors: List[str] = Field(default_factory=list, description="Wrong word blocks")

class TextResponseQuizData(BaseModel):
    """Used for Translate (CN<->EN/VI) and Writing"""
    source_text: Optional[str] = Field(None, description="Text to translate or prompt for writing")
    accepted_answers: List[str] = Field(default_factory=list, description="Valid variations of the translation")
    is_strict: bool = False

class SpeakingQuizData(BaseModel):
    """Used for Speaking"""
    expected_text: str
    expected_pinyin: Optional[str] = None
    time_limit: int = 15

# --- Core Models ---

class GeneratedQuiz(BaseModel):
    quiz_type: Literal[
        'Flashcard', 
        'Single Choice', 'Multiple Choice', 'Image Selection', 
        'Fill in the Blank', 'Translate to CN', 'Translate from CN', 'Writing',
        'Listening', 'Speaking', 
        'Match Pairs', 'Order Sentence'
    ]
    
    question_en: str
    question_vi: str
    explanation_en: Optional[str] = None
    explanation_vi: Optional[str] = None
    
    # Media prompts at the Quiz level
    audio_url: Optional[str] = None
    image_url: Optional[str] = None
    
    order: int
    
    # Polymorphic fields: Map the Enum type to the correct data container
    # 1. Flashcard
    flashcard: Optional[FlashCardData] = None
    
    # 2. Options (Single Choice, Multiple Choice, Image Selection, Listening)
    options: Optional[List[QuizOptionData]] = None
    
    # 3. Pairs (Match Pairs)
    pairs: Optional[List[MatchPairData]] = None
    
    # 4. Sentence (Fill in the Blank, Order Sentence)
    sentence: Optional[SentenceQuizData] = None
    
    # 5. Text Input (Translate..., Writing)
    text_response: Optional[TextResponseQuizData] = None
    
    # 6. Speaking (Speaking)
    speaking: Optional[SpeakingQuizData] = None

class GeneratedSection(BaseModel):
    # STRICTLY MATCHING YOUR LESSON SECTION TYPE ENUM
    section_type: Literal[
        'Vocabulary', 'Grammar', 'Reading', 'Listening', 
        'Speaking', 'Conversation', 'Final Test'
    ]
    title_en: str
    title_vi: str
    content_md: Optional[str] = Field(None, description="Markdown theory/dialogue text")
    order: int
    quizzes: List[GeneratedQuiz] = Field(..., description="List of quizzes in this section")

class GeneratedLessonContent(BaseModel):
    sections: List[GeneratedSection]
    
class CritiqueEvaluation(BaseModel):
    approved: bool = Field(description="Set to True if the content is perfect for the HSK level. False if changes are needed.")
    feedback: str = Field(description="Specific instructions on what to fix (e.g., 'The word 'Economy' is too hard for HSK1', 'Distractors in Q3 are too obvious').")
    
class SectionBlueprint(BaseModel):
    """The Architect's plan for a single section."""
    section_type: Literal['Vocabulary', 'Grammar', 'Reading', 'Listening', 'Speaking', 'Conversation', 'Final Test']
    title_en: str
    title_vi: str
    teaching_goal: str = Field(..., description="Pedagogical goal (e.g., 'Introduce basic greetings').")
    required_concepts: List[str] = Field(..., description="Specific words or grammar points to cover in this section.")
    recommended_quiz_types: List[str] = Field(..., description="Which quiz types best fit this section (e.g., 'Flashcard', 'Match Pairs').")

class LessonBlueprint(BaseModel):
    """Output of Agent 1"""
    rationale: str = Field(..., description="Reasoning for this lesson's structure.")
    sections: List[SectionBlueprint]

In [ ]:
llm = ChatOpenAI(model="gpt-5-mini", streaming=True,temperature=0.7, api_key="") # GPT-4o is recommended for complex JSON

# --- Setup LLMs ---
# Planner needs to be smart (GPT-4)
planner_llm = llm
# Writer needs to be creative but structured (GPT-4)
writer_llm = llm
# Reviewer needs to be strict (GPT-4)
critic_llm = llm


# --- Define State ---
class AgentState(TypedDict):
    # Inputs
    course_info: dict
    lesson_info: dict
    
    # Internal State
    lesson_plan: Optional[LessonBlueprint]       # Output of Agent 1
    draft_content: Optional[GeneratedLessonContent] # Output of Agent 2
    
    # Feedback Loop
    critique_feedback: Optional[str]             # Output of Agent 3
    revision_count: int 
    is_approved: bool


In [91]:
# --- 3. AGENT 1: The Architect (Planner) ---
def architect_node(state: AgentState):
    course = state['course_info']
    lesson = state['lesson_info']
    
    parser = PydanticOutputParser(pydantic_object=LessonBlueprint)
    
    # FIX: Explicitly list allowed types and FORBID "Practice"
    system_msg = """
    You are a Senior HSK Curriculum Architect. 
    Your job is to PLAN the structure of a lesson. Do not write the quizzes yet.
    
    ### STRICT ENUM RULES
    You must ONLY use the following `section_type` values:
    - 'Vocabulary' (For Flashcards)
    - 'Grammar' (For Theory)
    - 'Reading' (For Text comprehension practice)
    - 'Listening' (For Audio comprehension practice)
    - 'Speaking' (For Pronunciation)
    - 'Conversation' (For Dialogue)
    - 'Final Test' (For General Review/Mixed Practice)
    
    **CRITICAL:** Do NOT use "Practice" as a section type. If you want a practice section, use 'Reading', 'Listening', or 'Final Test'.

    ### INSTRUCTIONS
    1. Analyze the Learning Objectives.
    2. Must define 7-8 Sections (Scaffolding: Acquisition -> Application -> Production).
    3. Explain WHY each section is needed in the `rationale`.
    4. List specific concepts (words/grammar) required for each section.
    """
    
    user_msg = """
    Course: {course_name} ({hsk_level})
    Lesson: {lesson_title}
    Objectives: {objectives}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("user", user_msg),
        ("user", "\nOutput Blueprint JSON:\n{format_instructions}")
    ])
    
    formatted_inputs = {
        "course_name": course['name'],
        "hsk_level": course['hsk_level'],
        "lesson_title": lesson['title'],
        "objectives": lesson['learning_objectives'],
        "format_instructions": parser.get_format_instructions()
    }
    
    # Add error handling to retry if it fails (optional but recommended)
    try:
        response = planner_llm.invoke(prompt.invoke(formatted_inputs))
        return {"lesson_plan": parser.parse(response.content)}
    except Exception as e:
        # If the Architect fails, we can't proceed. In a production app, you might loop here.
        # For now, we return empty which will stop the graph gracefully or throw.
        print(f"Architect Error: {e}") 
        raise e

In [92]:
# --- AGENT 2: The Content Creator (Writer) ---
def writer_node(state: AgentState):
    # Inputs
    plan = state['lesson_plan']
    feedback = state.get('critique_feedback')
    revisions = state.get('revision_count', 0)
    
    parser = PydanticOutputParser(pydantic_object=GeneratedLessonContent)
    
    system_msg = """
    You are an Expert Content Writer for a Chinese Learning App (HSK Standard).
    Your goal is to turn a high-level Lesson Blueprint into concrete JSON data.

    ### GENERAL GUIDELINES
    1. **Strict Adherence:** Must follow the `lesson_plan` exactly. If the plan asks for "Order Sentence" for "你好", do not create a "Flashcard".
    2. **Language Level:** Use *only* vocabulary appropriate for the target HSK level.
    3. **Bilingual:** All explanations/definitions must be in both English (`_en`) and Vietnamese (`_vi`).
    4. **Quantity:** Must have at least 18-20 quizzes for a section.

    ### QUIZ TYPE SPECIFICATIONS (STRICT RULES)

    **1. 'Flashcard' (Vocabulary Only)**
       - **Usage:** Introducing new words.
       - **Data:** Fill `flashcard` field.
       - **Requirements:** - `word_hanzi`: The character(s).
         - `pinyin`: The pronunciation in Pinyin.
         - `definition_en`: Concise English meaning.
         - `definition_vi`: Concise Vietnamese meaning.
         - `example_sentence`: A simple sentence (mostly known words + this new word).
         - `example_pinyin`: Pinyin for the example sentence.
         - `example_meaning_en`: English meaning of the example sentence.
         - `example_meaning_vi`: Vietnamese meaning of the example sentence.
         - `extra_info_md`: Optional extra info in Markdown (e.g., cultural notes).

    **2. 'Single Choice' (Grammar/Reading)**
       - **Usage:** Standard testing.
       - **Data:** Fill `options` list.
       - **Structure:** Exactly **4 Options**.
       - **Logic:** 1 Option has `is_correct=True`, 3 Options have `is_correct=False`.
       - **Distractors:** Wrong answers must be grammatically plausible or visual lookalikes.

    **3. 'Multiple Choice' (Select All)**
       - **Usage:** When multiple answers are valid.
       - **Data:** Fill `options` list.
       - **Logic:** At least 2 Options must be `is_correct=True`.

    **4. 'Match Pairs' (Gamification)**
       - **Usage:** Testing Vocabulary recall.
       - **Data:** Fill `pairs` list.
       - **Structure:** Generate exactly at least **4 pairs** per quiz.
       - **Format:** `left_text` = Hanzi, `right_text` = Pinyin/Meaning.

    **5. 'Order Sentence' (Syntax Practice)**
       - **Usage:** Grammar practice.
       - **Data:** Fill `sentence` field.
       - **Logic:** Provide the **Full Correct Sentence** in `chinese_sentence`.
       - **Note:** The Frontend will handle the shuffling. You just provide the correct answer.

    **6. 'Fill in the Blank' (Context Practice)**
       - **Usage:** Testing specific vocabulary in context.
       - **Data:** Fill `sentence` field.
       - **Logic:** - `chinese_sentence`: The full sentence *including* the word to be removed.
         - `correct_answer`: The specific word that goes in the blank.
         - `distractors`: List of 3 incorrect words that fit grammatically but not semantically.

    **7. 'Listening' (Comprehension)**
       - **Usage:** Audio testing.
       - **Data:** Fill `options` list.
       - **Content:** The `question_en` should be "Listen and select the correct answer" or "What did you hear?".
       - **Note:** Leave `audio_url` empty (None) unless you have a specific placeholder URL strategy.

    **8. 'Speaking' (Pronunciation)**
       - **Usage:** User reads text aloud.
       - **Data:** Fill `speaking` field.
       - **Logic:** `expected_text` is the Hanzi the user must say. `time_limit` usually 10-15s.

    **9. 'Translate to CN' / 'Writing'**
       - **Usage:** Output/Production practice.
       - **Data:** Fill `text_response` field.
       - **Logic:** `accepted_answers` must be a list of valid string variations (e.g. ["Hello", "Hi"]).

    ### FORMATTING
    Output strict JSON matching the schema. Do not explain your code.
    """
    
    user_msg_template = """
    BLUEPRINT:
    {blueprint_json}
    """
    
    if feedback and revisions > 0:
        user_msg_template += "\n\n!!! CRITICAL FEEDBACK !!!\nReviewer Feedback: {feedback}\nFix issues and regenerate."

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("user", user_msg_template),
        ("user", "\nOutput Final Content JSON:\n{format_instructions}")
    ])
    
    # Prepare inputs safely
    inputs = {
        "blueprint_json": plan.model_dump_json(), # Pass JSON string as variable content
        "format_instructions": parser.get_format_instructions(),
        "feedback": feedback if feedback else ""
    }
    
    try:
        response = writer_llm.invoke(prompt.invoke(inputs))
        return {
            "draft_content": parser.parse(response.content),
            "revision_count": revisions + 1
        }
    except Exception as e:
        # If parsing fails, trigger a revision with the error as feedback
        return {"critique_feedback": f"JSON Parsing Error: {str(e)}", "revision_count": revisions + 1}

In [93]:
# --- 5. AGENT 3: The Reviewer (Birdbrain) ---
def reviewer_node(state: AgentState):
    draft = state.get('draft_content')
    course = state['course_info']
    
    if not draft:
        return {"is_approved": False, "critique_feedback": "No content generated."}
        
    parser = PydanticOutputParser(pydantic_object=CritiqueEvaluation)
    
    system_msg = f"""
    You are 'Birdbrain', a strict QA Auditor.
    
    CHECKLIST:
    1. HSK Compliance: Are words appropriate for {course['hsk_level']}?
    2. Distractor Quality: Are wrong answers plausible?
    3. Naturalness: Is the Chinese natural?
    4. Alignment: Does content match the Architect's Blueprint?
    
    Return 'approved': True ONLY if perfect.
    """
    
    user_msg_template = """
    DRAFT TO REVIEW:
    {draft_json}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("user", user_msg_template),
        ("user", "{format_instructions}")
    ])
    
    inputs = {
        "draft_json": draft.model_dump_json(),
        "format_instructions": parser.get_format_instructions()
    }
    
    response = critic_llm.invoke(prompt.invoke(inputs))
    eval_result = parser.parse(response.content)
    
    return {
        "is_approved": eval_result.approved,
        "critique_feedback": eval_result.feedback
    }

In [94]:
def should_continue(state: AgentState):
    if state['is_approved']:
        return "end"
    if state['revision_count'] > 3:
        return "end"
    return "revise"

In [95]:
workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("architect", architect_node)
workflow.add_node("writer", writer_node)
workflow.add_node("reviewer", reviewer_node)

# Add Edges
workflow.set_entry_point("architect")
workflow.add_edge("architect", "writer")
workflow.add_edge("writer", "reviewer")

workflow.add_conditional_edges(
    "reviewer",
    should_continue,
    {
        "end": END,
        "revise": "writer"
    }
)

app = workflow.compile()

In [ ]:
#Get the graph visualization in Mermaid format
print(app.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	architect(architect)
	writer(writer)
	reviewer(reviewer)
	__end__([<p>__end__</p>]):::last
	__start__ --> architect;
	architect --> writer;
	reviewer -. &nbsp;end&nbsp; .-> __end__;
	reviewer -. &nbsp;revise&nbsp; .-> writer;
	writer --> reviewer;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [98]:
def main():
    # 1. Load Data
    # Ensure these paths are correct relative to where you run the script
    if not os.path.exists('../input_data/lessons.csv'):
        print("Error: Input files not found. Check path '../input_data/'")
        return

    courses_df = pd.read_csv('../input_data/courses.csv') 
    lessons_df = pd.read_csv('../input_data/lessons.csv')

    # Data lists (Storage for CSV export)
    data_sections = []
    data_quizzes = []
    data_flashcards = []
    data_options = []
    data_pairs = []
    data_sentences = []
    data_speaking = []
    data_text_response = []
    
    section_id_counter = 1
    quiz_id_counter = 1

    print(f"Starting 3-Agent Generation Pipeline for {len(lessons_df)} lessons...")

    for _, lesson_row in lessons_df.iterrows():
        print(f"╔══════════════════════════════════════════════════════════════╗")
        print(f"║ Processing: {lesson_row['title']}")
        print(f"╚══════════════════════════════════════════════════════════════╝")
        
        # Find parent course info
        course_row = courses_df[courses_df['id'] == lesson_row['course_id']].iloc[0]

        # 2. Initial State for the 3-Agent Workflow
        initial_state = {
            "course_info": course_row.to_dict(),
            "lesson_info": lesson_row.to_dict(),
            
            # Agent 1 Output container
            "lesson_plan": None,
            
            # Agent 2 Output container
            "draft_content": None,
            
            # Agent 3 Feedback container
            "critique_feedback": None,
            
            # Loop Controls
            "revision_count": 0,
            "is_approved": False
        }

        # 3. Run the Graph
        try:
            for event in app.stream(initial_state):
                for node_name, state_update in event.items():
                    # Update our local state with the new data from the node
                    initial_state.update(state_update)
                    
                    # Visual Feedback
                    if node_name == "architect":
                        pprint.pprint(f"\n[Blueprint Created] 🏗️  Rationale: {state_update['lesson_plan'].rationale}...")
                    
                    elif node_name == "writer":
                        rev = initial_state.get('revision_count', 0)
                        pprint.pprint(f"\n[Draft Generated] ✍️  Revision #{rev}")
                    
                    elif node_name == "reviewer":
                        approved = state_update.get('is_approved')
                        feedback = state_update.get('critique_feedback')
                        if approved:
                            pprint.pprint(f"\n ✅[Review Passed] Birdbrain approved the content!")
                        else:
                            pprint.pprint(f"\n ❌[Review Failed] Feedback: {feedback}")
                            pprint.pprint("   ↪️  Looping back to Writer...")

        except Exception as e:
            print(f"❌ Graph Error: {e}")
            continue

        # 4. Extract Final Content
        # We grab 'draft_content' because that is the final JSON produced by the Writer
        content: GeneratedLessonContent = initial_state.get('draft_content')
        
        if not content:
            print(f"FAILED to generate content for {lesson_row['title']}")
            continue

        pprint.pprint(f"✅ Content Approved after {initial_state['revision_count']} revisions.")
        
        # 5. Flatten Data for CSV Export
        lesson_key = lesson_row['title'] 
        
        for sec in content.sections:
            current_section_id = section_id_counter 
            section_id_counter += 1
            
            # Note: Using 'title_vi' to match your Pydantic Schema and Django Model
            data_sections.append({
                "id": current_section_id, 
                "lesson_key": lesson_key, 
                "section_type": sec.section_type, 
                "title_en": sec.title_en, 
                "title_vi": sec.title_vi, 
                "content_md": sec.content_md, 
                "order": sec.order
            })

            for quiz in sec.quizzes:
                current_quiz_id = quiz_id_counter
                quiz_id_counter += 1
                
                data_quizzes.append({
                    "id": current_quiz_id, 
                    "section_id": current_section_id, 
                    "quiz_type": quiz.quiz_type, 
                    "question_en": quiz.question_en, 
                    "question_vi": quiz.question_vi, 
                    "audio_url": quiz.audio_url,
                    "image_url": quiz.image_url, 
                    "explanation_en": quiz.explanation_en, 
                    "explanation_vi": quiz.explanation_vi, 
                    "order": quiz.order
                })

                # --- Handle Polymorphic Data ---
                if quiz.flashcard:
                    data_flashcards.append({"quiz_id": current_quiz_id, **quiz.flashcard.model_dump()})
                
                if quiz.options:
                    for opt in quiz.options:
                        data_options.append({"quiz_id": current_quiz_id, **opt.model_dump()})
                
                if quiz.pairs:
                    for pair in quiz.pairs:
                        data_pairs.append({"quiz_id": current_quiz_id, **pair.model_dump()})
                
                if quiz.sentence:
                    data_sentences.append({"quiz_id": current_quiz_id, **quiz.sentence.model_dump()})
                
                if quiz.speaking:
                    data_speaking.append({"quiz_id": current_quiz_id, **quiz.speaking.model_dump()})
                
                if quiz.text_response:
                    dump = quiz.text_response.model_dump()
                    # Convert list to string for CSV compatibility
                    dump['accepted_answers'] = str(dump['accepted_answers'])
                    data_text_response.append({"quiz_id": current_quiz_id, **dump})

    # 6. Save to CSV
    os.makedirs('output_data', exist_ok=True)
    
    def save_csv(data, filename):
        if data: 
            pd.DataFrame(data).to_csv(f'output_data/{filename}', index=False)
            print(f"Saved {filename}")
        else:
            print(f"No data for {filename}")
    
    save_csv(data_sections, 'sections.csv')
    save_csv(data_quizzes, 'quizzes.csv')
    save_csv(data_flashcards, 'flashcards.csv')
    save_csv(data_options, 'options.csv')
    save_csv(data_pairs, 'pairs.csv')
    save_csv(data_sentences, 'sentences.csv')
    save_csv(data_speaking, 'speaking.csv')
    save_csv(data_text_response, 'text_response.csv')
    
    print("\nGeneration Complete. Check 'output_data/' folder.")

In [99]:
if __name__ == "__main__":
    main()

Starting 3-Agent Generation Pipeline for 2 lessons...
╔══════════════════════════════════════════════════════════════╗
║ Processing: Hello - Nǐ hǎo
╚══════════════════════════════════════════════════════════════╝
('\n'
 '[Blueprint Created] 🏗️  Rationale: This lesson is scaffolded from '
 'acquisition to production to ensure beginners internalize the five target '
 'objectives (nǐ hǎo, nǐ, hǎo, wǒ, jiào) in multiple modalities. 1) Vocabulary '
 'first builds form-meaning mappings for the five lexical items (characters, '
 'pinyin, tones) so learners can recognize and recall them. 2) Speaking '
 '(pronunciation) immediately after vocabulary trains accurate sound and tone '
 'production, preventing fossilized pronunciation errors early. 3) Listening '
 'gives learners receptive practice with native-speed realizations of '
 'greetings and introductions, reinforcing vocabulary in context. 4) Grammar '
 'explains the basic sentence pattern (subject + 叫 + name) and the fixed '
 'greeting 你好,